# 🗺️ Visualización de NDVI en Mapa Embebido

Este notebook permite visualizar la imagen NDVI generada en Google Earth Engine en un mapa interactivo después de exportarla a Google Drive.

## Características:
- Visualización interactiva con zoom y pan
- Paleta de colores personalizada para NDVI
- Estadísticas del NDVI (promedio, mín, máx)
- Exportación a Google Drive
- Generación de mapa HTML para compartir

In [1]:
# 📦 Importar librerías necesarias
import ee
import geemap

In [2]:
# 🔐 Autenticación e inicialización de Google Earth Engine
# Descomenta la siguiente línea si es la primera vez que usas GEE
# ee.Authenticate()

ee.Initialize(project='agtech-suscripci-1759332935027')

In [3]:
# 🌍 Definir área de interés y fechas

# Área de interés (Turipaná, Córdoba)
aoi = ee.Geometry.Rectangle([-75.89, 8.88, -75.86, 8.90])

# Fechas de análisis
start_date = '2022-01-01'
end_date = '2024-03-31'

print(f"📍 Área de interés: Turipaná, Córdoba")
print(f"📅 Período: {start_date} a {end_date}")

📍 Área de interés: Turipaná, Córdoba
📅 Período: 2022-01-01 a 2024-03-31


In [4]:
# 🛰️ Cargar y procesar imágenes MODIS NDVI

# Colección MODIS NDVI
modis_ndvi = ee.ImageCollection('MODIS/061/MOD13Q1') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi) \
    .select('NDVI')

# Validar si hay imágenes disponibles
image_count = modis_ndvi.size().getInfo()
print(f"✅ Número de imágenes encontradas: {image_count}")

if image_count > 0:
    # Calcular NDVI promedio
    ndvi_composite = modis_ndvi.mean().clip(aoi)
    
    # Escalar NDVI de [0-10000] a [-1, 1]
    ndvi_scaled = ndvi_composite.multiply(0.0001)
    
    print("✅ NDVI calculado correctamente")
else:
    print("❌ No se encontraron imágenes MODIS NDVI")

✅ Número de imágenes encontradas: 52
✅ NDVI calculado correctamente


In [5]:
# 📊 Calcular estadísticas del NDVI

stats = ndvi_scaled.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.minMax(),
        sharedInputs=True
    ),
    geometry=aoi,
    scale=250,
    maxPixels=1e9
).getInfo()

print("\n📊 Estadísticas del NDVI:")
print(f"   NDVI Promedio: {stats.get('NDVI_mean', 'N/A'):.3f}")
print(f"   NDVI Mínimo: {stats.get('NDVI_min', 'N/A'):.3f}")
print(f"   NDVI Máximo: {stats.get('NDVI_max', 'N/A'):.3f}")

# Interpretación
ndvi_mean = stats.get('NDVI_mean', 0)
if ndvi_mean < 0.2:
    print("\n🔍 Interpretación: Vegetación escasa o suelo desnudo")
elif ndvi_mean < 0.4:
    print("\n🔍 Interpretación: Vegetación moderada o pastizales")
elif ndvi_mean < 0.6:
    print("\n🔍 Interpretación: Vegetación saludable")
else:
    print("\n🔍 Interpretación: Vegetación muy densa y saludable")


📊 Estadísticas del NDVI:
   NDVI Promedio: 0.642
   NDVI Mínimo: 0.564
   NDVI Máximo: 0.699

🔍 Interpretación: Vegetación muy densa y saludable


In [6]:
# 🗺️ Crear mapa interactivo

# Crear mapa centrado en el área de interés
Map = geemap.Map()

# Obtener coordenadas del centro del área
center = aoi.centroid().coordinates().getInfo()
Map.setCenter(center[0], center[1], 13)

# Paleta de colores para NDVI
# Rojo: vegetación pobre/suelo desnudo
# Amarillo: vegetación moderada
# Verde: vegetación densa/saludable
ndvi_palette = [
    '#d73027',  # Rojo oscuro (NDVI bajo)
    '#f46d43',  # Rojo
    '#fdae61',  # Naranja
    '#fee08b',  # Amarillo claro
    '#d9ef8b',  # Verde amarillento
    '#a6d96a',  # Verde claro
    '#66bd63',  # Verde
    '#1a9850',  # Verde oscuro (NDVI alto)
]

# Parámetros de visualización
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ndvi_palette
}

# Agregar capa NDVI al mapa
Map.addLayer(ndvi_scaled, vis_params, 'NDVI MODIS (Promedio)')

# Agregar contorno del área de interés
Map.addLayer(aoi, {'color': 'blue'}, 'Área de Interés')

# Agregar barra de colores/leyenda
Map.add_colorbar(vis_params, label='NDVI', layer_name='NDVI MODIS (Promedio)')

# Agregar herramienta de inspección
Map.add_layer_manager()

print("✅ Mapa creado exitosamente")

AttributeError: module 'ee.data' has no attribute '_credentials'

In [ ]:
# 💾 Exportar imagen a Google Drive

task = ee.batch.Export.image.toDrive(
    image=ndvi_composite,
    description='MODIS_NDVI_EneMar2024',
    folder='GEE_Exports',
    fileNamePrefix='NDVI_MODIS_Turipana_Marzo2024',
    region=aoi,
    scale=250,
    crs='EPSG:4326',
    maxPixels=1e13
)

task.start()

print("💾 Exportación iniciada a Google Drive")
print(f"   📁 Carpeta: GEE_Exports")
print(f"   📄 Archivo: NDVI_MODIS_Turipana_Marzo2024")
print(f"\n⏳ Estado de la tarea: {task.status()}")
print("\nℹ️  Puedes verificar el progreso en: https://code.earthengine.google.com/tasks")

In [ ]:
# 🖼️ Mostrar mapa interactivo

print("🗺️  Mapa interactivo NDVI - Turipaná")
print("   ✨ Puedes hacer zoom, mover el mapa y hacer clic para ver valores NDVI")
print("   📊 Usa el inspector de capas para más detalles")

# Mostrar el mapa
Map

In [ ]:
# 💾 Guardar mapa como HTML (opcional)

output_html = "mapa_ndvi_turipana.html"
Map.to_html(filename=output_html, title="Mapa NDVI - Turipaná")

print(f"✅ Mapa guardado como: {output_html}")
print(f"   Puedes abrir este archivo en cualquier navegador web para compartir")

## 📝 Notas adicionales:

### Interpretación de valores NDVI:
- **NDVI < 0.2**: Suelo desnudo, rocas, agua
- **NDVI 0.2 - 0.4**: Vegetación escasa, pastizales secos
- **NDVI 0.4 - 0.6**: Vegetación moderada, cultivos
- **NDVI 0.6 - 0.8**: Vegetación densa, bosques
- **NDVI > 0.8**: Vegetación muy densa

### Paleta de colores:
- 🔴 Rojo: Baja actividad fotosintética
- 🟡 Amarillo: Actividad fotosintética moderada
- 🟢 Verde: Alta actividad fotosintética

### Aplicaciones en silvopastoreo:
- Identificar zonas de bajo vigor vegetativo
- Monitorear la salud de los pastizales
- Planificar rotación de ganado
- Detectar áreas que necesitan intervención

In [7]:
# 🔍 OPCIONAL: Análisis temporal - Comparar con período anterior

# Definir período anterior para comparación
start_date_prev = '2021-01-01'
end_date_prev = '2022-03-31'

# Calcular NDVI del período anterior
modis_ndvi_prev = ee.ImageCollection('MODIS/061/MOD13Q1') \
    .filterDate(start_date_prev, end_date_prev) \
    .filterBounds(aoi) \
    .select('NDVI')

if modis_ndvi_prev.size().getInfo() > 0:
    ndvi_prev = modis_ndvi_prev.mean().clip(aoi).multiply(0.0001)
    
    # Calcular diferencia
    ndvi_diff = ndvi_scaled.subtract(ndvi_prev)
    
    # Estadísticas de la diferencia
    diff_stats = ndvi_diff.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=250,
        maxPixels=1e9
    ).getInfo()
    
    print("\n📈 Análisis temporal:")
    print(f"   Diferencia promedio NDVI: {diff_stats.get('NDVI', 0):.3f}")
    
    if diff_stats.get('NDVI', 0) > 0:
        print("   ✅ Mejora en la cobertura vegetal")
    else:
        print("   ⚠️ Disminución en la cobertura vegetal")
    
    # Visualizar diferencia
    Map_diff = geemap.Map()
    Map_diff.setCenter(center[0], center[1], 13)
    
    diff_vis = {
        'min': -0.3,
        'max': 0.3,
        'palette': ['red', 'white', 'green']
    }
    
    Map_diff.addLayer(ndvi_diff, diff_vis, 'Cambio NDVI')
    Map_diff.addLayer(aoi, {'color': 'blue'}, 'Área de Interés')
    
    print("\n🗺️ Mapa de cambio NDVI (rojo = disminución, verde = aumento):")
    display(Map_diff)


📈 Análisis temporal:
   Diferencia promedio NDVI: -0.018
   ⚠️ Disminución en la cobertura vegetal


AttributeError: module 'ee.data' has no attribute '_credentials'